### Setup VAE
#### DEPRECATED

In [2]:
from data_utils.dataloaders import ZeoData
from data_utils.utils import collate_fn, get_cif_str
from source.data_utils.crystal_utils import build_crystal
from basic_VAE import BasicVAE
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
import tqdm

In [3]:
zeo_data = ZeoData(list(range(100)), 14.919, n_samples=100)

100%|██████████| 100/100 [00:00<00:00, 195.31it/s]


In [5]:
cif_str = get_cif_str(f'../data/MOR_dataloader_test_100/RHO_clusters_0.cif')
crystal = build_crystal(cif_str, primitive=True)


c:\Users\dglav\Anaconda3\envs\thesis\Lib\site-packages\pymatgen\io\cif.py:687: UserWarning: A 1-line symmetry op P1 CIF is detected!
  warnings.warn(msg)
c:\Users\dglav\Anaconda3\envs\thesis\Lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: A 1-line symmetry op P1 CIF is detected!
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [12]:

print(crystal.get_space_group_info())
print(crystal.atomic_numbers)

('Cm', 8)
(13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8)


### Train VAE

In [3]:
trainset = ZeoData(list(range(0,80)), 14.919, n_samples=80)
testset = ZeoData(list(range(80,100)), 14.919, n_samples=20)

lr = 0.01
hid_size = 32
z_size = 32
beta = 0.1
adam_beta1 = 0.9
adam_beta2 = 0.99
batch_size = 16

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

100%|██████████| 20/20 [00:00<00:00, 1538.35it/s]


In [4]:
vae = BasicVAE(torch.tensor([14.919, 14.919, 14.919]), hidden_size=hid_size, latent_size=z_size).cuda()
opt = Adam(vae.parameters(), lr=lr, betas=(adam_beta1, adam_beta2))
epochs = 100

In [8]:
batch = next(iter(trainloader))

In [9]:
batch


EdgeDataBatch(x=[768], edge_index=[2, 10752], pos=[768, 3], batch=[768], ptr=[17])

In [5]:
losses = []
for e in range(epochs):
    vae.train()
    n_batches = len(trainloader)
    pbar = tqdm.tqdm(enumerate(trainloader))
    for idx, databatch in pbar:
        log_dict = vae.training_step(databatch, beta=beta, opt=opt)
        pbar.set_postfix_str(f"loss: {log_dict['loss']:.3f}")
        losses.append(log_dict['loss'])
        step = e*n_batches + idx

400it [00:32, 12.35it/s, loss: 0.015] 
400it [00:30, 13.28it/s, loss: 0.018] 
400it [00:30, 13.23it/s, loss: -0.007]
400it [00:32, 12.44it/s, loss: 0.015] 
400it [00:31, 12.88it/s, loss: 0.008] 
400it [00:29, 13.52it/s, loss: 0.024] 
400it [00:28, 13.83it/s, loss: 0.007] 
400it [00:29, 13.75it/s, loss: 0.020] 
400it [00:29, 13.42it/s, loss: 0.009] 
400it [00:29, 13.67it/s, loss: -0.004]
400it [00:29, 13.64it/s, loss: 0.025] 
400it [00:28, 13.97it/s, loss: 0.020] 
400it [00:30, 13.25it/s, loss: 0.009] 
400it [00:28, 13.83it/s, loss: 0.020] 
400it [00:26, 15.19it/s, loss: 0.017] 
400it [00:26, 14.87it/s, loss: 0.014] 
400it [00:29, 13.44it/s, loss: 0.017] 
400it [00:29, 13.49it/s, loss: 0.014] 
400it [00:28, 13.82it/s, loss: 0.013] 
400it [00:29, 13.50it/s, loss: 0.016] 
400it [00:29, 13.74it/s, loss: 0.018] 
400it [00:29, 13.41it/s, loss: 0.025] 
400it [00:29, 13.51it/s, loss: -0.004]
400it [00:29, 13.65it/s, loss: 0.008] 
400it [00:28, 13.81it/s, loss: 0.010] 
400it [00:28, 14.16it/s, 